# ДЗ2: Основи аналізу та алгоритмічні стратегії (Теми 1–2)

Цей ноутбук містить готове рішення двох завдань:
1. **Min/Max** (Divide & Conquer) — функція `minmax_divide_and_conquer` повертає `(min, max)` за **O(n)**.
2. **Оптимізація черги 3D-принтера** (Greedy) — функція `optimize_printing` з використанням `@dataclass`.


In [1]:
# Монтування Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception as e:
    print('Google Colab не виявлено або Drive недоступний у цій середовищі.')


Mounted at /content/drive


## Завдання 1 — Min/Max (Divide & Conquer)

In [2]:
from typing import List, Tuple

def minmax_divide_and_conquer(arr: List[float]) -> Tuple[float, float]:
    """Повертає (min, max) для масиву arr за допомогою підходу 'розділяй і володарюй'. O(n)."""
    if not arr:
        raise ValueError("Порожній масив недопустимий")

    def helper(lo: int, hi: int) -> Tuple[float, float]:
        if lo == hi:
            x = arr[lo]
            return (x, x)
        if hi == lo + 1:
            a, b = arr[lo], arr[hi]
            return (a, b) if a <= b else (b, a)
        mid = (lo + hi) // 2
        left_min, left_max = helper(lo, mid)
        right_min, right_max = helper(mid + 1, hi)
        return (left_min if left_min <= right_min else right_min,
                left_max if left_max >= right_max else right_max)

    return helper(0, len(arr) - 1)

# Швидка перевірка
arr = [7, -2, 15, 0, 9, -5, 3]
print("Min/Max:", minmax_divide_and_conquer(arr))  # Очікується (-5, 15)


Min/Max: (-5, 15)


## Завдання 2 — Оптимізація черги 3D-принтера (Greedy + dataclass)

In [3]:
from typing import List, Dict
from dataclasses import dataclass

@dataclass
class PrintJob:
    id: str
    volume: float
    priority: int  # 1 = найвищий
    print_time: int  # хвилини

@dataclass
class PrinterConstraints:
    max_volume: float
    max_items: int

def optimize_printing(print_jobs: List[Dict], constraints: Dict) -> Dict:
    cons = PrinterConstraints(**constraints)
    if cons.max_items <= 0 or cons.max_volume <= 0:
        raise ValueError("Обмеження принтера мають бути додатними.")

    jobs = [PrintJob(**j) for j in print_jobs]
    for j in jobs:
        if j.volume <= 0 or j.print_time <= 0 or j.priority not in (1, 2, 3):
            raise ValueError(f"Некоректні дані задачі: {j}")
        if j.volume > cons.max_volume:
            raise ValueError(f"Модель '{j.id}' має об'єм, що перевищує max_volume принтера.")

    # Стабільне сортування: 1→2→3, зі збереженням відносного порядку в межах пріоритету
    jobs_with_index = list(enumerate(jobs))
    jobs_with_index.sort(key=lambda p: (p[1].priority, p[0]))
    remaining = [j for _, j in jobs_with_index]

    print_order = []
    total_time = 0

    while remaining:
        highest_prio = min(j.priority for j in remaining)

        batch = []
        used_items = 0
        used_volume = 0.0

        # 1) Спершу — задачі найвищого доступного пріоритету
        i = 0
        while i < len(remaining) and used_items < cons.max_items:
            j = remaining[i]
            if j.priority == highest_prio and j.volume + used_volume <= cons.max_volume:
                batch.append(j)
                used_items += 1
                used_volume += j.volume
                remaining.pop(i)
            else:
                i += 1

        # 2) Добиваємо батч нижчими пріоритетами (2, потім 3), якщо є місткість
        if used_items < cons.max_items and used_volume < cons.max_volume and remaining:
            for pr in (2, 3):
                if pr < highest_prio:
                    continue
                i = 0
                while i < len(remaining) and used_items < cons.max_items:
                    j = remaining[i]
                    if j.priority == pr and j.volume + used_volume <= cons.max_volume:
                        batch.append(j)
                        used_items += 1
                        used_volume += j.volume
                        remaining.pop(i)
                    else:
                        i += 1

        batch_time = max(job.print_time for job in batch)
        total_time += batch_time
        print_order.extend([job.id for job in batch])

    return {"print_order": print_order, "total_time": total_time}


## Тести з умови

In [4]:
def test_printing_optimization():
    # Тест 1: Однаковий пріоритет
    test1_jobs = [
        {"id": "M1", "volume": 100, "priority": 1, "print_time": 120},
        {"id": "M2", "volume": 150, "priority": 1, "print_time": 90},
        {"id": "M3", "volume": 120, "priority": 1, "print_time": 150}
    ]

    # Тест 2: Різні пріоритети
    test2_jobs = [
        {"id": "M1", "volume": 100, "priority": 2, "print_time": 120},  # лабораторна
        {"id": "M2", "volume": 150, "priority": 1, "print_time": 90},   # дипломна
        {"id": "M3", "volume": 120, "priority": 3, "print_time": 150}   # особистий проєкт
    ]

    # Тест 3: Перевищення обмежень (на групу)
    test3_jobs = [
        {"id": "M1", "volume": 250, "priority": 1, "print_time": 180},
        {"id": "M2", "volume": 200, "priority": 1, "print_time": 150},
        {"id": "M3", "volume": 180, "priority": 2, "print_time": 120}
    ]

    constraints = {"max_volume": 300, "max_items": 2}

    print("Тест 1 (однаковий пріоритет):")
    result1 = optimize_printing(test1_jobs, constraints)
    print(f"Порядок друку: {result1['print_order']}")
    print(f"Загальний час: {result1['total_time']} хвилин")

    print("\nТест 2 (різні пріоритети):")
    result2 = optimize_printing(test2_jobs, constraints)
    print(f"Порядок друку: {result2['print_order']}")
    print(f"Загальний час: {result2['total_time']} хвилин")

    print("\nТест 3 (перевищення обмежень):")
    result3 = optimize_printing(test3_jobs, constraints)
    print(f"Порядок друку: {result3['print_order']}")
    print(f"Загальний час: {result3['total_time']} хвилин")

# Запуск тестів
test_printing_optimization()


Тест 1 (однаковий пріоритет):
Порядок друку: ['M1', 'M2', 'M3']
Загальний час: 270 хвилин

Тест 2 (різні пріоритети):
Порядок друку: ['M2', 'M1', 'M3']
Загальний час: 270 хвилин

Тест 3 (перевищення обмежень):
Порядок друку: ['M1', 'M2', 'M3']
Загальний час: 450 хвилин
